# Aprentissage supervisé

* 1: [Loading the database](#step1)
* 2: [Calculation of all the image's signature](#step2)
* 3: [Calculation of the centers of each class(boundaries)](#step3)
* 4: [Testing the classifier](#step4)

<a id="step1"></a>
## Loading the database 

#### For the classifier: we will use 700 images for training and 210 for testing and cross-k validation after, saving each metric calculated
#### For the neural network: We will use 700 images for training, 105 for validation and 105 for testing. 

In [1]:
# function to read all the database files and its expected classification
# input: path to the csv file
# output: a list with all the images filenames
def load_database(path):
    file_names = []
    targets = []
    f = open(path, 'r')
    
    #split all the lines into a list
    list_files = f.read().split('\n')
    
    #scan all these lines
    for files in list_files:
        #split each line with a space, we get = [filename, target]
        full_line = files.split()
        
        if(len(full_line) != 0):   
            file_names.append(full_line[0])
            targets.append(int(full_line[1]))
            #print(full_line[0], full_line[1], i)
    
    f.close()
    return file_names, targets

In [2]:
filenames, targets = load_database("../FruitLearning/Resources/targets.csv")
print("Number of images: ", len(filenames))

Number of images:  910


<a id='step2'></a>
## Calculation of all the image's signature

In [3]:
import cv2
import tree_utils as tu
from tqdm import tqdm

signatures = []
for file in filenames:#tqdm(filenames):#filenames:
    #print(file)
    img = cv2.imread(file)
    if(img is None):
        print(file)
    else:
        #sig_1 = tu.get_hist(tu.to_HSV(img))
        sig_1 = tu.get_hist(img)
        sig_2 = tu.get_LBP(tu.get_Luminance(img))
        signatures.append([sig_1, sig_2])

In [43]:
print(signatures[0][1].shape)

(1, 64)


In [4]:
import numpy as np
def signature_mean(list_sig):
    sig_hist = [sig[0] for sig in list_sig]
    sig_lbp = [sig[1] for sig in list_sig]
    
    return [np.mean(np.array(sig_hist), axis=0), np.mean(np.array(sig_lbp), axis=0)]
        

In [5]:
print(len(signatures))

910


In [58]:
print(signatures[0][0])
print(signatures[1][0])
vec = [signatures[0], signatures[1]]
print(signature_mean(vec))

[[7.33583490e-03 1.64329268e-02 3.16838649e-03 0.00000000e+00
  7.58442777e-03 6.89985929e-02 1.76360225e-03 0.00000000e+00
  3.27861163e-03 7.17940901e-02 2.60787992e-03 0.00000000e+00
  3.08864916e-03 4.72467167e-02 1.84099437e-03 0.00000000e+00
  1.88015947e-02 4.51031895e-02 1.45895872e-02 0.00000000e+00
  3.56707317e-03 1.00234522e-01 1.10131332e-02 0.00000000e+00
  3.65853659e-04 5.36585366e-02 1.24366792e-02 0.00000000e+00
  4.69043152e-06 1.14845216e-02 3.15666041e-03 0.00000000e+00
  8.12617261e-03 5.56472795e-02 1.33419325e-02 0.00000000e+00
  1.30628518e-03 6.88320826e-02 2.03447467e-02 0.00000000e+00
  2.11069418e-05 1.18878987e-02 7.55863039e-03 0.00000000e+00
  0.00000000e+00 3.09568480e-04 4.92495310e-05 0.00000000e+00
  5.48710131e-02 7.09615385e-02 1.40220450e-02 0.00000000e+00
  1.08363039e-01 4.09333959e-02 1.26242964e-02 0.00000000e+00
  0.00000000e+00 9.84990619e-04 2.57973734e-04 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]]
[[3.53579812e-03 5.45

In [7]:
all_index_spring_sig = [ i for i,f in enumerate(filenames) if 'spring' in f]
all_spring_sig = [signatures[index] for index in all_index_spring_sig]
print(len(all_spring_sig))

all_index_winter_sig = [i for i,f in enumerate(filenames) if 'winter' in f]
all_winter_sig = [signatures[index] for index in all_index_winter_sig]
print(len(all_winter_sig))

all_index_autumn_sig = [i for i,f in enumerate(filenames) if 'autumn' in f]
all_autumn_sig = [signatures[index] for index in all_index_autumn_sig]

all_index_summer_sig = [i for i,f in enumerate(filenames) if 'summer' in f]
all_summer_sig = [signatures[index] for index in all_index_summer_sig]

246
224


In [8]:
centroid_spring = signature_mean(all_spring_sig)
centroid_winter = signature_mean(all_winter_sig)
centroid_autumn = signature_mean(all_autumn_sig)
centroid_summer = signature_mean(all_summer_sig)

In [46]:
right = 0
coef = [0.7, 0.3]
for i,sig in enumerate(signatures):
    dist_spring = tu.distance(centroid_spring, sig, coef)
    dist_winter = tu.distance(centroid_winter, sig, coef)
    dist_autumn = tu.distance(centroid_autumn, sig, coef)
    dist_summer = tu.distance(centroid_summer, sig, coef)
    #prediction spring = 0, summer = 1, autumn = 2, winter = 3
    
    if(dist_spring <= dist_winter and dist_spring <= dist_autumn and dist_spring <= dist_summer):
        prediction = 0
    elif(dist_summer <= dist_winter and dist_summer <= dist_spring and dist_summer <= dist_autumn):
        prediction = 1
    elif(dist_autumn <= dist_winter and dist_autumn <= dist_spring and dist_autumn <= dist_summer):
        prediction = 2
    else:
        prediction = 3
    
    #check if prediction was good
    if(prediction == targets[i]):
        right += 1
    else:
        print('TARGET:',targets[i])
        print('distance spring:',dist_spring)
        print('distance summer:',dist_summer)
        print('distance autumn:', dist_autumn)
        print('distance winter:',dist_winter)
        

print(right)
print('accuracy:', right/len(targets))

TARGET: 3
distance spring: 0.5985588352488992
distance summer: 0.6043382877044404
distance autumn: 0.6404600281546413
distance winter: 0.606101746625889
TARGET: 0
distance spring: 0.5837266956280581
distance summer: 0.5626487422812217
distance autumn: 0.6148662220881864
distance winter: 0.6590208267702137
TARGET: 2
distance spring: 0.4653345468846362
distance summer: 0.4571100812906403
distance autumn: 0.47924924088078374
distance winter: 0.5281303910105534
TARGET: 1
distance spring: 0.5906171716312552
distance summer: 0.597000048385603
distance autumn: 0.6330482608592324
distance winter: 0.6032767885653185
TARGET: 2
distance spring: 0.5382962724408675
distance summer: 0.5216422882986388
distance autumn: 0.5408809880031786
distance winter: 0.5955050912825989
TARGET: 2
distance spring: 0.5786444599748652
distance summer: 0.5984726434061053
distance autumn: 0.5828422996529125
distance winter: 0.5872364726062037
TARGET: 0
distance spring: 0.5011497029591887
distance summer: 0.498311033135

distance autumn: 0.5008818983446418
distance winter: 0.6541391560402285
TARGET: 3
distance spring: 0.5218860253029389
distance summer: 0.516049231560933
distance autumn: 0.5728134772810258
distance winter: 0.6014316475837691
TARGET: 2
distance spring: 0.4794700212978864
distance summer: 0.43991195812157474
distance autumn: 0.4759124558636644
distance winter: 0.5268922314238786
TARGET: 2
distance spring: 0.5010502129323038
distance summer: 0.48893436430340753
distance autumn: 0.5016128903084123
distance winter: 0.5884987979827334
TARGET: 2
distance spring: 0.6105742952731296
distance summer: 0.6219731947555052
distance autumn: 0.6396892948184371
distance winter: 0.6144615453215455
TARGET: 3
distance spring: 0.542318104374656
distance summer: 0.5428488974742034
distance autumn: 0.5715625731728995
distance winter: 0.6137075052517026
TARGET: 0
distance spring: 0.5162311670880125
distance summer: 0.504219333486914
distance autumn: 0.5432972514164373
distance winter: 0.5933637466673786
TARGE

distance summer: 0.7221529131712573
distance autumn: 0.7191785251000743
distance winter: 0.7145769888376259
TARGET: 2
distance spring: 0.6153966076650262
distance summer: 0.611005568057074
distance autumn: 0.6232610200441844
distance winter: 0.6063951498996762
TARGET: 3
distance spring: 0.5975200131490912
distance summer: 0.6202654017315267
distance autumn: 0.6417172169615677
distance winter: 0.6456639759713432
TARGET: 1
distance spring: 0.5519915792479874
distance summer: 0.5356724724858649
distance autumn: 0.4856625567970259
distance winter: 0.6377881633174398
TARGET: 2
distance spring: 0.5867859139460766
distance summer: 0.594960507783946
distance autumn: 0.5753280578457773
distance winter: 0.4879465728067494
TARGET: 2
distance spring: 0.5934196606326575
distance summer: 0.5993651748174177
distance autumn: 0.5913083865154601
distance winter: 0.5853518444762069
TARGET: 3
distance spring: 0.5966566690263212
distance summer: 0.6442581337409321
distance autumn: 0.6582692688395636
distan

distance summer: 0.6492577078465179
distance autumn: 0.6500339197894499
distance winter: 0.6397935321748853
TARGET: 2
distance spring: 0.5791378571040854
distance summer: 0.5859125658521944
distance autumn: 0.5655271556808308
distance winter: 0.5419110871108267
TARGET: 0
distance spring: 0.5479365513224551
distance summer: 0.520507951831312
distance autumn: 0.5650909541837
distance winter: 0.6423413161445756
TARGET: 3
distance spring: 0.628095920414347
distance summer: 0.6308060193774601
distance autumn: 0.6677865318861396
distance winter: 0.6555478566453927
TARGET: 3
distance spring: 0.7148835074771883
distance summer: 0.7209886133797208
distance autumn: 0.7218225685506328
distance winter: 0.7154462521170162
TARGET: 3
distance spring: 0.5343584580687281
distance summer: 0.5532384946040794
distance autumn: 0.5741746050559651
distance winter: 0.6180101375012411
TARGET: 2
distance spring: 0.5070870298773814
distance summer: 0.4821004098714484
distance autumn: 0.5242909818060402
distance 

In [5]:
right = 0
coef = [0.8, 0.2]
for i,sig_1 in enumerate(signatures):
    d_min = 10000
    index = -1
    for j,sig in enumerate(signatures):
        if(i == j):
            continue
        else:
            dist = tu.distance(sig_1, sig, coef)
            #prediction spring = 0, summer = 1, autumn = 2, winter = 3
            if(dist < d_min):
                d_min = dist
                index = j

    #check if prediction was good
    prediction = targets[index]
    if(prediction == targets[i]):
        right += 1
    else:
        print('TARGET:',targets[i])
        #print('distance spring:',dist_spring)
        #print('distance summer:',dist_summer)
        #print('distance autumn:', dist_autumn)
        #print('distance winter:',dist_winter)
        

print(right)
print('accuracy:', right/len(targets))

TARGET: 0
TARGET: 1
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 1
TARGET: 1
TARGET: 1
TARGET: 1
TARGET: 0
TARGET: 2
TARGET: 0
TARGET: 1
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 1
TARGET: 1
TARGET: 3
TARGET: 1
TARGET: 0
TARGET: 1
TARGET: 0
TARGET: 3
TARGET: 3
TARGET: 3
TARGET: 3
TARGET: 1
TARGET: 0
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 2
TARGET: 0
TARGET: 1
TARGET: 2
TARGET: 0
TARGET: 2
TARGET: 0
TARGET: 1
TARGET: 2
TARGET: 0
TARGET: 0
TARGET: 2
TARGET: 0
TARGET: 1
TARGET: 3
TARGET: 0
TARGET: 1
TARGET: 0
TARGET: 1
TARGET: 2
TARGET: 3
TARGET: 3
TARGET: 1
TARGET: 0
TARGET: 3
TARGET: 0
TARGET: 1
TARGET: 3
TARGET: 0
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 1
TARGET: 1
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 1
TARGET: 2
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 3
TARGET: 2
TARGET: 1
TARGET: 1
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 1
TARGET: 1
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 3
TARGET: 1


In [33]:
print('Reference:',filenames[0])

print('Winter test:',filenames[1])
print('Spring test:',filenames[2])
print('Summer test:',filenames[3])
print('Autumn test:',filenames[4])

#import tree_utils as tu
dist_winter = tu.distance(centroid_winter, signatures[1], [0.8, 0.2])
print('Distante to a winter:',dist)
dist_spring = tu.distance(centroid_spring, signatures[1], [0.8, 0.2])
print('Distante to a spring:',dist)
dist_summer = tu.distance(centroid_summer, signatures[1], [0.8, 0.2])
print('Distante to a summer:',dist)
dist_autumn = tu.distance(centroid_autumn, signatures[1], [0.8, 0.2])
print('Distante to a autumn:',dist)
if(dist_spring <= dist_winter and dist_spring <= dist_autumn and dist_spring <= dist_summer):
    prediction = 0
elif(dist_summer <= dist_winter and dist_summer <= dist_spring and dist_summer <= dist_autumn):
    prediction = 1
elif(dist_autumn <= dist_winter and dist_autumn <= dist_spring and dist_autumn <= dist_summer):
    prediction = 2
else:
    prediction = 3

print(prediction)
print(targets[1])

if(prediction == int(targets[1])):
    right += 1

print(right)

Reference: ../FruitLearning/Resources/database/spring/137.png
Winter test: ../FruitLearning/Resources/database/winter/726.png
Spring test: ../FruitLearning/Resources/database/spring/175.png
Summer test: ../FruitLearning/Resources/database/summer/406.png
Autumn test: ../FruitLearning/Resources/database/autumn/622.png
Distante to a winter: 0.5850568501035276
Distante to a spring: 0.5850568501035276
Distante to a summer: 0.5850568501035276
Distante to a autumn: 0.5850568501035276
3
3
663


In [62]:
dist = tu.distance(centroid_spring, signatures[1], [0.8, 0.2])
print('Distante to a winter:',dist)
dist = tu.distance(centroid_spring, signatures[2], [0.8, 0.2])
print('Distante to a spring:',dist)
dist = tu.distance(centroid_spring, signatures[3], [0.8, 0.2])
print('Distante to a summer:',dist)
dist = tu.distance(centroid_spring, signatures[4], [0.8, 0.2])
print('Distante to a autumn:',dist)

Distante to a winter: 0.43050714858488
Distante to a spring: 0.3094622777727072
Distante to a summer: 0.39922652928334057
Distante to a autumn: 0.6052329905561256


In [14]:
from keras.preprocessing import image
img = cv2.imread('../FruitLearning/Resources/database/autumn/511.jpg')
if(img is None):
    print("noen")
#tensor_3d = image.img_to_array(img)
#tensor_3d.shape